# Preparing the DC2 Run 1.1p Data

**Owner:** Phil Marshall (@drphilmarshall)

**Last Run:** 2018-10-24 (by @drphilmarshall)

**Goals:** Use the DC2 Run 1.1p truth catalog, and DRP Object table, to construct a design matrix `X` (from the truth table) and corresponding response variables `y` (from the DRP Object table).

**Notes:** This notebook was made by adapting the following notebooks:
* Scott Daniel's DC2 Tutorial [`truth_gcr_intro.ipynb`](https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/truth_gcr_intro.ipynb)
* Yao-Yuan Mao's DC2 Tutorial [`matching_fof.ipynb`](https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/matching_fof.ipynb)

To run this notebook, follow the instructions to setup Jupyter-dev at NERSC: https://confluence.slac.stanford.edu/x/1_ubDQ

## Setup

In [ ]:
# import packages and methods that will be used in this notebook

import healpy
import numpy as np
import pandas as pd
import GCRCatalogs
from astropy.coordinates import SkyCoord
import FoFCatalogMatching

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

## Loading the Input Truth

In [ ]:
# truth_catalog = GCRCatalogs.load_catalog('dc2_truth_run1.1_static')
truth_catalog = GCRCatalogs.load_catalog('dc2_truth_run1.1', {'md5': None})

You can list the quantities contained in the catalog using the same API as any other GCR-based catalog.

In [ ]:
all_true = truth_catalog.list_all_quantities(include_native=True)
all_true

The `get_quantity_info` method will give you access to descriptions of all of the native quantities in the catalog.  Note that the quantities `ugrizy` map directly to `mag_true_[ugrizy]'.

In [ ]:
for qty in all_true:
    info_dict = truth_catalog.get_quantity_info(qty)
    print(qty,info_dict)

Let's define a small photometric and spatial subset to work with, just for demo purposes.

In [ ]:
some_true = ['u',
             'g',
             'r',
             'i',
             'z',
             'y',
             'ra',
             'dec',
             'star',
             'object_id',
             'redshift',
             'healpix_2048'
]

print(len(some_true))

We'll want to keep our training and test set small while developing, but then increase its size later. A good option for selecting a subset of objects is to filter by sky position, either relative to some reference point, or using the healpixellation.

In the cells below, we will query the catalog for all of the bright stars and galaxies in a 0.2 degree disc centered on `RA=55.8`, `Dec=-28.8`.

In the next cell, we will define some methods (`filter_on_healpix` and `filter_on_dist`) needed to make that selection.  These methods will be passed into the catalog's `get_quantities` method with the `filters` kwarg.  The methods should accept numpy arrays and return a numpy array of booleans indicating whether or not each object in the input arrays passed the test.

We will use `healpy`'s `query_disc` method to find which healpixels overlap our region of interest.

In [ ]:
center_ra = 55.8
center_dec = -28.8
radius = 0.2

ra_min, ra_max = center_ra - radius, center_ra + radius
dec_min, dec_max = center_dec - radius, center_dec + radius

center_ra_rad = np.radians(center_ra)
center_dec_rad = np.radians(center_dec)

center_vec = np.array([np.cos(center_dec_rad)*np.cos(center_ra_rad),
                       np.cos(center_dec_rad)*np.sin(center_ra_rad),
                       np.sin(center_dec_rad)])

list_of_healpix = healpy.query_disc(2048, center_vec, np.radians(radius), nest=True, inclusive=True)

def filter_on_healpix(hp):
    return np.array([hh in list_of_healpix for hh in hp])

def angularSeparation(ra, dec, center_ra, center_dec):
    return SkyCoord(ra, dec, unit='deg').separation(SkyCoord(center_ra, center_dec, unit='deg')).deg

def filter_on_dist(ra, dec):
    return angularSeparation(ra, dec, center_ra, center_dec) < radius

coord_filters = [
    'ra >= {}'.format(ra_min),
    'ra < {}'.format(ra_max),
    'dec >= {}'.format(dec_min),
    'dec < {}'.format(dec_max),
]

Now we let's query the catalog for all objects in the region of interest with magnitude `r<27.0`.  This query functions like any other GRC-based catalog query with one exception.  The truth catalog is ultimately stored as a sqlite database.  This means that all `native_filters` (filters applied directly to the catalog's native quantities), should be phrased as strings which could be inserted into an SQL `WHERE` clause.  Because the `native_filters` are applied when the quantities loaded into memory (as opposed to `filters`, which are applied after the quantities have been loaded), we want these to be as restrictive as possible so as to limit the memory consumption of the loaded catalog.  The sqlite databases are indexed on `star`, `agn`, `sprinkled`, `healpix_2048` and `object_id`.

**Note:** We are aware that `nside==2048` healpixels may be too fine a resolution to helpfully limit the catalog query.  We are open to the idea of using a coarser resolution in future truth catalogs.

After the coarse spatial limits applied by the `native_filter` on `healpix_2048`, we use the `filter_on_healpix` and `filter_on_dist` methods to actually get sources in our region of interest.

In [ ]:
true_objects = truth_catalog.get_quantities(some_true,
                                            native_filters=['r<27.0',
                                                            'healpix_2048<=%d' % list_of_healpix.max(),
                                                            'healpix_2048>=%d' % list_of_healpix.min()],
                                            filters=[(filter_on_healpix, 'healpix_2048')]+coord_filters)  
                                            
#                                            filters=[(filter_on_healpix, 'healpix_2048'),
#                                                     (filter_on_dist, 'ra', 'dec')])

In [ ]:
print(len(true_objects['ra']))

In [ ]:
plt.scatter(true_objects['ra'], true_objects['dec'], alpha=0.01, color="blue")
plt.xlim(ra_max, ra_min)
plt.ylim(dec_min, dec_max)
plt.xlabel('RA / deg')
plt.ylabel('Dec / deg');

Now we will perform a similar spatial query for objects which were not added by the sprinkler (`sprinkled==0`) with magnitudes `r<27`.

> Nb: "the sprinkler" is a piece of code used to add an extra population of AGN and supernovae into the Utral Deep Drilling Field of DC2.  Objects added by the sprinkler will have no counterpart in the underlying protoDC2 extragalactic catalog, which is why we have added a `sprinkled` flag to the catalog.

In [ ]:
unsprinkled_true_objects = truth_catalog.get_quantities(some_true,
                                                        native_filters=['sprinkled==0',
                                                                           'r<27.0',
                                                                           'healpix_2048<=%d' % list_of_healpix.max(),
                                                                           'healpix_2048>=%d' % list_of_healpix.min()],
                                                        filters=[(filter_on_healpix, 'healpix_2048'),
                                                                 (filter_on_dist, 'ra', 'dec')])

In [ ]:
print(len(unsprinkled_true_objects['ra']))

Only a handful of objects in this 0.2 deg radius circular aperture have been sprinkled.

## Loading the Observed LSST Objects

Now we need to load in a DRP Object table, so that we can (later) match its contents (spatially) to the input truth objects. We'll do this for just a single tract, no. 4850.

In [ ]:
observed_catalog = GCRCatalogs.load_catalog('dc2_coadd_run1.1p_tract4850')

Let's first visually inspect the footprint of one tract of the coadd catalog.
When `return_iterator` is turned on, the method `get_quantities` will return an 
iterator, and each element in the iterator will be the quantities we requested in 
different chunks of the dataset. 

For coadd catalogs, the different chunks happen to be different patches, 
resulting in a different color for each patch in the scatter plot below.

In [ ]:
all_observed = observed_catalog.list_all_quantities(include_native=True)
len(all_observed)

The object table contains a lot of quantities... Let's define a small photometric and spatial subset to use. We'll make this bigger than the truth subset though, because we expect to be emulating more quantities than we have truth values for, in general.

In [ ]:
sorted(all_observed[1300:1320])

In [ ]:
some_observed = ['mag_u_cModel',
                 'mag_g_cModel',
                 'mag_r_cModel',
                 'mag_i_cModel',
                 'mag_z_cModel',
                 'mag_y_cModel',
                 'ra',
                 'dec',
                 'extendedness',
                 'objectId',
                 'ext_shapeHSM_HsmShapeRegauss_e1',
                 'ext_shapeHSM_HsmShapeRegauss_e2',
                 'ext_shapeHSM_HsmShapeRegauss_sigma',
                 'u_modelfit_CModel_fracDev',
                 'u_modelfit_CModel_flux',
                 'g_modelfit_CModel_fracDev',
                 'g_modelfit_CModel_flux',
                 'r_modelfit_CModel_fracDev',
                 'r_modelfit_CModel_flux',
                 'i_modelfit_CModel_fracDev',
                 'i_modelfit_CModel_flux',
                 'z_modelfit_CModel_fracDev',
                 'z_modelfit_CModel_flux',
                 'y_modelfit_CModel_fracDev',
                 'y_modelfit_CModel_flux'
]

print(len(some_observed))

Let's choose a small RA and Dec range to do the matching so that it won't take too long! Let's also define a magnitude cut.

> Nb. Recall that we selected objects in the truth catalog using the following circular aperture:
```
center_ra = 54.6
center_dec = -28.0
radius = 0.2
```

In [ ]:
mag_filters = [
    (np.isfinite, 'mag_r_cModel'),
    'mag_r_cModel < 27.0',
]

# For some reason this filter doesn't work, here.
# coord_filters = [
#     (filter_on_dist, 'ra', 'dec')
# ]

coord_filters = [
    'ra >= {}'.format(ra_min),
    'ra < {}'.format(ra_max),
    'dec >= {}'.format(dec_min),
    'dec < {}'.format(dec_max),
]

In [ ]:
# Load some observed quantities, using both of the filters we just defined. 
observed_objects = observed_catalog.get_quantities(some_observed, 
                                                filters=(mag_filters + coord_filters))

In [ ]:
print(len(observed_objects['ra']))

In [ ]:
plt.scatter(observed_objects['ra'], observed_objects['dec'], alpha=0.01, color="red")
plt.xlim(ra_max, ra_min)
plt.ylim(dec_min, dec_max)
plt.xlabel('RA / deg')
plt.ylabel('Dec / deg');

## Matching the True and Observed Objects

We now have an array of observed object measurements, and an array of true object parameters: next we need to match them up, by position on the sky. 

`FoFCatalogMatching.match` takes a dictionary of catalogs to match, a friends-of-friends linking length. Because our "catalog" is not an astropy table or pandas dataframe, `len(truth_coord)` won't give the actual length of the table
so we need to specify `catalog_len_getter` so that the code knows how to get the length of the catalog.

In [ ]:
results = FoFCatalogMatching.match(
    catalog_dict={'truth': true_objects, 'observed': observed_objects},
    linking_lengths=1.0,
    catalog_len_getter=lambda x: len(x['ra']),
)

Now we want to count the number of truth and coadd objects *for each group*
but instead of looping over groups, we can do this in a smart (and very fast) way. First we need to know which rows are from the truth catalog and which are from the coadd.

In [ ]:
truth_mask = results['catalog_key'] == 'truth'
observed_mask = ~truth_mask

# Then, np.bincount will give up the number of id occurrences 
# (like histogram but with integer input):
n_groups = results['group_id'].max() + 1
n_true = np.bincount(results['group_id'][truth_mask], minlength=n_groups)
n_observed = np.bincount(results['group_id'][observed_mask], minlength=n_groups)

# Now n_true and n_observed are the number of true/observed objects 
# in each group, and we want to make a 2d histogram of (n_true, n_observed). 
n_max = max(n_true.max(), n_observed.max()) + 1
hist_2d = np.bincount(n_observed * n_max + n_true, minlength=n_max*n_max).reshape(n_max, n_max)

plt.imshow(np.log10(hist_2d+1), extent=(-0.5, n_max-0.5, -0.5, n_max-0.5), origin='lower');
plt.xlabel('Number of true objects');
plt.ylabel('Number of observed objects');
plt.colorbar(label=r'$\log(N_{\rm groups} \, + \, 1)$');

In [ ]:
one_to_one = hist_2d[1,1]
total = np.sum(hist_2d)
print(one_to_one, " out of ", total, " FoF groups are 1-to-1 matches.")

About half of the FoF groups are 1-to-1 matches. Some of this is due to the mis-orientation between the healpixel we used to speed up the truth catalog pre-selection, but at 27th magnitude, there's also a lot of blending. 

## Formatting into `X` and `y`

Now that we have a set of matched objects, we can reformat them into a design matrix `X` and the corresponding response variables `y`. We'll make these as simple `numpy` arrays.

Let's focus on the objects in the groups that have a 1-to-1 true/observed match. 

In [ ]:
one_to_one_group_mask = np.in1d(results['group_id'], np.flatnonzero((n_true == 1) & (n_observed == 1)))

We need the row indices in the *original* truth/observed 
catalogs for those 1-to-1 groups.

In [ ]:
truth_idx = results['row_index'][one_to_one_group_mask & truth_mask]
observed_idx = results['row_index'][one_to_one_group_mask & observed_mask]

Now we can use these indices to pull out the data arrays, and make `pandas` data frames from them.

In [ ]:
true_matched = pd.DataFrame(true_objects).iloc[truth_idx].reset_index(drop=True)
observed_matched = pd.DataFrame(observed_objects).iloc[observed_idx].reset_index(drop=True)

In [ ]:
true_matched.head()

In [ ]:
observed_matched.head()

Any machine learning engine worth its salt must be able to accept `pandas.DataFrame` objects as input, both for `X` and `y`. So, lets finally associate the true catalog with the input design matrix `X`, and the observed catalog with the output response variables `y`. And then let's just double check the shape of the arrays.

In [ ]:
X, y = true_matched, observed_matched

In [ ]:
X.shape, y.shape

## Summary

We loaded in the truth table and observed DRP Object table from Run 1.1p, and matched the objects in a small (0.4x0.4 degree) patch of sky. This gave as about 20k 1-to-1 matches, which we packaged up ready for machine learning training and testing. 

A good next step is given to us in the final section [`matching_fof.ipynb` DC2 tutorial](https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/matching_fof.ipynb): joining to the extragalactic catalog that the truth table was made from, and bringing in some additional true object properties to better predict the observed properties. We'll leave that for another time.

## Appendix

Let's re-do all of the above work using the `derp.Emulator`.

In [ ]:
import sys
sys.path.insert(0, '/global/homes/p/pjm43/desc/derp')

In [ ]:
import derp

emulator = derp.Emulator()

some_true = ['u','g','r','i','z','y','ra','dec','star','object_id','redshift','healpix_2048']
some_observed = ['mag_u_cModel','mag_g_cModel','mag_r_cModel','mag_i_cModel','mag_z_cModel','mag_y_cModel','ra','dec','extendedness','objectId','ext_shapeHSM_HsmShapeRegauss_e1','ext_shapeHSM_HsmShapeRegauss_e2','ext_shapeHSM_HsmShapeRegauss_sigma','u_modelfit_CModel_fracDev','u_modelfit_CModel_flux','g_modelfit_CModel_fracDev','g_modelfit_CModel_flux','r_modelfit_CModel_fracDev','r_modelfit_CModel_flux','i_modelfit_CModel_fracDev','i_modelfit_CModel_flux','z_modelfit_CModel_fracDev','z_modelfit_CModel_flux','y_modelfit_CModel_fracDev','y_modelfit_CModel_flux']

X,y = emulator.make_training_set(truth='dc2_truth_run1.1', 
                                 observed='dc2_coadd_run1.1p_tract4850',
                                 region=(55.8, -28.8, 0.2), 
                                 true_quantities=some_true, 
                                 observed_quantities=some_observed)

In [ ]:
X.head()

In [ ]:
y.head()